In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

import pickle

In [2]:
most_recent_extract = '2023-06-11 16:25'
df = pd.read_csv(f'data/reddit_posts_raw_{most_recent_extract}.csv')

In [3]:
df.shape

(1971, 6)

In [4]:
df.head(1)

,subreddit,id,created_utc,title,selftext,top_comment_text
0,dating,1471ube,2023-06-11 18:49:33,Am I Clueless?,So there is this girl I’ve known my whole life...,NaN


#### Baseline

> The majority class holds 50.63% of responses.  This is the baseline score to beat.

> Even class distribution makes a 75/25 train test split possible.

In [5]:
df['subreddit'].value_counts(normalize = True)

datingoverthirty    0.506342
dating              0.493658
Name: subreddit, dtype: float64

### Self-text only

In [6]:
X = pd.Series(df['selftext'])
y = df['subreddit'].map({'dating': 0,
                    'datingoverthirty':1})

### Self Text and Top Comment - Alternative Path

In [7]:
# df['self_text_and_comment'] = df['self_text'].astype(str) + df['top_comment_text'].astype(str)
# X = pd.Series(df['self_text_and_comment'])
# y = df['subreddit'].map({'dating': 0,
#                    'datingoverthirty':1})

### Train-Test Split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42)
X_train.to_pickle('./pickled_models/X_train.pkl')
X_test.to_pickle('./pickled_models/X_test.pkl')
y_train.to_pickle('./pickled_models/y_train.pkl')
y_test.to_pickle('./pickled_models/y_test.pkl')

## Baseline Investigation with Standard Vectorizers

#### CountVectorizer

In [22]:
cvec0 = CountVectorizer() #standard CountVectorizer
cvec0.fit(X_train)
pickle.dump(cvec0, open('./pickled_models/cvec0_baseline', 'wb'))

> See Model Investigaion for Investigations

#### Tf-Idf Vectorizer

In [23]:
tvec0 = TfidfVectorizer()
pickle.dump(tvec0, open('./pickled_models/tvec0_baseline', 'wb'))

### Instantiate Vectorizers

In [5]:
# CountVectorizer
cvec = CountVectorizer()
# TF-IDF Term Frequency - Inverse Document Frequency Vectorizer
tvec = TfidfVectorizer()

### Intstantiate Evaluators

In [ ]:
#Logistic Regression
logr = LogisticRegression()
#Multinomial Naive Bayes
nb = MultinomialNB()